# Generate noise series for the production cost of generators

This notebook generates noise series for all 770 dispatchable generators of the European model.

In [1]:
import os, sys
sys.path.append('../TimeSeries/')

import time_series as ts

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Get a list of gens

Import the network file:

In [3]:
with open('../models/europe.json') as f:
    network = json.load(f)

Count the number of generators:

In [4]:
n_gens_all = len(network['gen'])
n_gens_all

815

Subtract non-dispatchable generators:

In [5]:
n_nuc = len(pd.read_csv('data/nuclear_2016.csv'))
n_nuc

44

In [6]:
n_gens = n_gens_all - n_nuc
n_gens

771

### Generate noise series

To be relevant, the noise on the generation cost must be of the same order of magnitude as the line cost computed in the model, which is directly related to the power flowing on the line.

We choose therefore a noise with a standard deviation of 0.1 in per-unit, corresponding to 10 MW.

In [7]:
noise_time_series = 0.1 * ts.generate_noise(count=n_gens)

In [8]:
noise_time_series.mean(), noise_time_series.std(axis=1).mean()

(2.62927320485265e-16, 0.09881085819277959)

Pack everything into a dataframe, with time steps as columns and generators as rows:

In [9]:
df = pd.DataFrame(noise_time_series)
df.index.name = 'id'
df

,0,1,2,3,4,5,6,7,8,9,...,8726,8727,8728,8729,8730,8731,8732,8733,8734,8735
id,,,,,,,,,,,,,,,,,,,,,
0,-0.026249,0.019820,0.079053,-0.043354,-0.097841,-0.049243,0.063898,0.011228,0.049185,0.065321,...,-0.028627,-0.037443,-0.095100,0.042149,-0.002588,-0.054946,0.074796,0.004821,-0.066613,0.032703
1,-0.070787,-0.126037,-0.219750,-0.191466,-0.253986,-0.225655,-0.229588,-0.131912,-0.114484,-0.208030,...,-0.103238,-0.194308,-0.176896,-0.125430,-0.227190,-0.259487,-0.243443,-0.170083,-0.225453,-0.282577
2,0.105176,0.069375,0.194085,0.116418,0.002906,0.080336,0.091157,0.143337,0.060922,0.069877,...,0.142099,0.123536,0.035663,0.062893,0.098023,0.204582,0.123608,0.127434,0.072822,0.022119
3,-0.022202,-0.053400,-0.150654,-0.126552,-0.105592,-0.080403,-0.006417,-0.042203,-0.098554,-0.074391,...,0.033301,0.067993,-0.018634,-0.078930,-0.048672,-0.116823,-0.127959,0.005478,-0.014501,-0.087602
4,-0.031232,-0.071578,-0.036415,-0.097197,-0.174624,-0.000073,-0.083693,-0.072326,0.072452,-0.141222,...,-0.020356,-0.072810,-0.028818,-0.048489,-0.014588,0.032067,0.038577,-0.048118,-0.083813,-0.040613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,0.051497,0.037597,-0.040690,-0.023936,0.008849,0.067003,-0.025712,-0.124639,0.046461,0.023628,...,-0.091343,-0.061776,0.067013,0.127442,0.082025,0.066766,0.148871,0.124395,0.041724,0.013287
767,0.007719,0.019191,0.093154,0.074431,0.033013,0.047951,0.061405,0.020297,0.047191,-0.072745,...,0.174553,0.081927,-0.093476,0.040415,0.107836,0.068557,0.135343,0.096640,0.061444,0.088013
768,0.054634,0.023580,0.023700,0.059767,0.012784,0.015650,0.068521,-0.029785,-0.077296,0.053626,...,0.058537,0.003009,0.018658,0.108853,0.046410,0.009881,0.014628,0.005796,0.027517,0.038485


Export to CSV format:

In [10]:
df.to_csv('data/cost_noise.csv', mode='x', index=False)

As with the loads, the whole procedure can be iterated:

In [11]:
# for label in range(0,10):
#     df = pd.DataFrame(0.1 * ts.generate_noise(count=n_gens), index=gen_ids)
#     df.index.name = 'id'
#     filename = "data/cost_noise_%02d.csv" % label
#     print("Writing to file '%s'..." % filename, end='')
#     df.to_csv(filename, mode='x')
#     print("ok")